![alt text](https://drive.google.com/uc?export=view&id=1DXUVHxd4t15mfuqMgMCLnsP4jWVI5EWz)
<br>
© Copyright The University of New South Wales - CRICOS 00098G

# Laboratory 2, Part B:
# Diabetes Hospitalisations - Linear Prediction

![alt text](https://drive.google.com/uc?export=view&id=105SGqeyo8RgLhSO8mN7ZE5OsG0YiLPKt)



---





---



# 1. Introduction
Following the visualisation and manipulation steps (data mining and machine learning work-flow), we are now moving to steps 5-7. Check this tutorial for more information:
http://scikit-learn.org/stable/tutorial/basic/tutorial.html



---



<b>Goal/Research question:</b> <font color=green> <b> our final goal is to build a predictive algorithm to predict readmission to hospital 30 days after discharge. </b></font>




---



## 1.1. Regression with L1-norm Regularization (Lasso) and L2-norm Regularization (Ridge)

In this exercise, we are going to fit a logistic regression to our data set, by using two techniques that constrains (or regularizes) the coefficient estimates with a L1-norm (Lasso regularization) and a L2-norm regularization (Ridge regularization).

1. The Lasso regularization will shrink the coefficient estimates towards zero or <b>directly to zero</b>. For this last reason, Lasso regularization <font color=green><b>could also be</b></font> a technique for <font color=green><b>feature selection </b></font>(those features that are not zero in the model).

2. The Ridge regularization will shrink the coefficient estimates **towards zero**.




---



## 1.2. Aims:
 1. To build a predictive model <b>using logistic regression with L1-norm and L2-norm regularization of the coefficient estimates</b> (Lasso and Ridge regularization).
 2. **To  choose evaluation metrics.**
 3. To manipulate features: standardization.
 4. To use training and test sets.
 5. To continue becoming familiar with the diabetes inpatient hospital dataset and the clinical terms contained in it.

It aligns with all the learning outcomes of our course:


1. Distinguish a range of task specific machine learning techniques appropriate for Health Data Science.
2. Design machine learning tasks for Health Data Science scenarios.
3. Apply machine learning workflow to health data problems.
4. Generate knowledge via the application of machine learning techniques to health data.




---





---



# 2. Initial Docstring:

All programs should have an initial docstring comment. It must include at least the following elements:

* Purpose: what is the aim of your code?
* Date created
* Author
* Date modified
* Author of the modification
* Method: how did you go about solving the problem?
* Data dictionary: The data dictionary should contain all the important variables and constants defined, their datatype (float, string, int) and a short description of what they are.
* List and defintions of functions: similar to the data dictionary, but with functions.
* List of libraries: libraries used in the program and their functionality.

Is there anything else you think we should include in the docstring? Please comment in the comments section of this week's laboratory.

Please read these two documents:
1. pandas docstring guide: https://pandas.pydata.org/pandas-docs/version/0.23/contributing_docstring.html
2. Style guide: https://www.cse.unsw.edu.au/~en1811/resources/style.html


<b> Docstring:</b>
#####################################################################################################################

(double-click here)


#####################################################################################################################



---





---



# 3. Dataset


In [ ]:
# Insert your comments and explanations

import sys
import numpy as np
import pandas as pd
import seaborn as sns
# We can also plot directly using pandas:
# https://pandas.pydata.org/docs/reference/plotting.html

In [ ]:
# Insert your comments and explanations

# Mount Google Drive
# We do not need to run this cell if you are not running this notebook in Google Colab

if 'google.colab' in str(get_ipython()):
    from google.colab import drive # import drive from Gogle colab
    root = '/content/drive'     # default location for the drive
    # print(root)                 # print content of ROOT (Optional)
    drive.mount(root)
else:
    print('Not running on CoLab')

Mounted at /content/drive


In [ ]:
# Insert your comments and explanations

from pathlib import Path

if 'google.colab' in str(get_ipython()):
    # EDIT THE PROJECT PATH IF DIFFERENT WITH YOUR ONE
    # You may need to change 'MyDrive' to 'My Drive'.
    project_path = Path(root) / 'MyDrive' / 'Colab Notebooks'

    # OPTIONAL - set working directory according to your google drive project path
    # import os
    # Change directory to the location defined in project_path
    # os.chdir(project_path)
else:
    project_path = Path()

In [ ]:
# Insert your comments and explanations

# import pickle
pickle_data_path = Path(project_path) /'hospital_final.pickle'
hospital = pd.read_pickle(pickle_data_path)

In [ ]:
hospital.sex_Female

0         True
1        False
2         True
3         True
4        False
         ...  
69265     True
69266     True
69267    False
69268    False
69269    False
Name: sex_Female, Length: 69267, dtype: bool

In [ ]:
hospital.columns



---





---



# 4. Feature scaling

For some of the methods covered in this chapter, specifically ridge and lasso regression, it is advisable to scale the features.

We can use several methods to that, for example:

1.  https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler

2. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In this example, we are going to use the `StandardScaler()`.

**Interpretation of beta coefficients**:

Keep in mind that the interpretation of regression coefficients becomes more complicated after the features are standardized. For instance, consider multiple linear regression. Prior to standardization, a unit change in $X_j$ corresponds to a change of $\hat \beta_j$ to the predicted response. Simple. However, after standardization of $X_j$, the interpretation of the coefficients becomes: for every increase of <b>one standard deviation</b> in $X_j$, the predicted response changes by $\hat \beta_j$ standard deviations. Still manageable, but not as straight forward.<p>

First, we split the hospital data set into two DataFrames: the features, stored in X, and the target, stored in y.

In [ ]:
# Insert your comments and explanations

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Insert your comments and explanations

X = hospital.drop(['readmission'], axis = 1)
y = hospital[['readmission']].values



---





---



### Use of `Stratify = y`

Now let's split the data into a training and test set. We will include the optional argument `stratify = y` to preserve the ratio between readmission=NO to readmission=YES.

In [ ]:
# Insert your comments and explanations

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size=0.25,random_state=0)
# stratify = y means we wish to preserve the ratio of y in the test and training sets

In [ ]:
# Insert your comments and explanations

# Sanity Check:

# Check that proportions are maintained:
y_test_df = pd.DataFrame(y_test)
y_test_df.columns = ['readmission']
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ['readmission']

print('*********************************************************************************')
print(f"""Number of records in original data frame (hospital): {len(y)},
Proportion of readmission in original data frame (hospital):\n {hospital.readmission.value_counts('NO')}\n
********************************************************************************* \n
Number of records in test set: {len(y_test)},
Proportion of readmission in test set:\n {y_test_df.readmission.value_counts('NO')}\n
********************************************************************************* \n
Number of records in training set: {len(y_train)},
Proportion of readmission in train set:\n {y_train_df.readmission.value_counts('NO')}
""")
print('*********************************************************************************')

We can see that the training and test set have maintained the proportion of NO:YES response from the original data.



---





---



### `StandardScaler()`

Now let's define the scaler we will use, which we decide is `StandardScaler()`. We can also easily replace one preprocessing algorithm with another by simply changing the scaler definition at this step. For instance, instead of using `StandardScaler()`, we could have used `MinMaxScaler`, or some other alternative.

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
# Insert your comments and explanations

scaler = StandardScaler()

### Feature scaling: How to apply scaling to our data.

Now we fit the scaler to the training data. It is important that we fit the scaler to **only the training set** and then apply that same scaler to the test set (remember that the test set is never seen until the very end, once our model has been fully trained). The purpose of having a test set is to mimic the situation where your model is making prediction decisions in the real world, when you do not have access to the true response. Therefore, we cannot use the test set for *anything* except comparing to predicted values. This means we cannot use the test set to fit the scaler.

Standardize features means to subtract the mean and divide by the standard deviation.
In this step, we calculate the actual means and variances for each feature in  <font color='green'> THE TRAIN SET.</font>
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
# Insert your comments and explanations

# Standardize features by removing the mean and scaling to unit variance.
# In this step, we calculate the actual means and variances for each feature in  THE TRAIN SET.
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler.fit(X_train)
# mu-training-set = mean of the age in the training set
# sigma-training-set = standard deviation of the age in the training set

To actually scale the data, use the transform method.

In [ ]:
# Insert your comments and explanations

# rescale the training data
X_train_scaled = scaler.transform(X_train)
# (X_train - mu-training-set )/sigma-training-set

In [ ]:
# Check
X_train_scaled

To apply predictive models to the scaled data, we also need to transform the test set.

In [ ]:
# Insert your comments and explanations

# scale the test data
X_test_scaled = scaler.transform(X_test)
# (X_test - mu-training-set )/sigma-training-set

In [ ]:
# Insert your comments and explanations

# Sanity check
X_test_scaled



---





---



# 5. Logistic Regression

## 5.1 Training a logistic regression model

Now we can train our logistic regression model using ridge regression. The default of logistic regression in <b>scikit-learn</b>  is to include a ridge penalty term in the linear component. The parameter controlling the regularization strength, <b>$alpha$</b>, or $C$ in our Python library. This parameter is also known as $\lambda$, and its relationship with $alpha$ is $(alpha =  C = \frac {1}{\lambda})$. <p>
    Small values of $C$ indicate strong regularization, whilst larger values of $C$ indicate weaker regularization. In fact, if we set $C$ to be an extremely large number, then $\lambda$ is effectively zero and the regression reduces to <b> regular logistic regression</b> .

In [ ]:
# Insert your comments and explanations

from sklearn.linear_model import LogisticRegression

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Let's use the 'liblinear' solver that is valid for L1 and L2 regularizations.

### Training a logistic regression model using L1-norm regularization (Lasso)

In [ ]:
# Insert your comments and explanations

# C=0.01 as a first step. We will see how to adjust this hyper-paremters next week
Log_Reg_L1 = LogisticRegression(C = 0.01 , penalty = 'l1', solver='liblinear').fit(X_train_scaled, y_train.ravel())
# ravel() used to convert y from column vector to 1d array, as required by the method

### Training a logistic regression model using L2-norm regularization (Ridge)

In [ ]:
# Insert your comments and explanations

# C=0.01 as a first step. We will see how to adjust this hyper-paremters next week
Log_Reg_L2 = LogisticRegression(C = 0.01 , penalty = 'l2',solver='liblinear').fit(X_train_scaled, y_train.ravel())
# ravel() used to convert y from column vector to 1d array, as required by the method



---





---



### <font color='blue'> Question 1: Print the beta coefficients of both logistic regression models and comment on your first impression. Change the C hyperparameter several times and print the beta coefficients again to see how they change.</font>

<p><font color='green'> Tip:  You can use the following structure: "ModelName.coef_" to print the beta coefficients only.</font></p>
<p><font color='green'> You can use the following structure: "coefficients = pd.concat([pd.DataFrame(X.columns,columns=['Features']), pd.DataFrame(np.transpose(ModelName.coef_),columns=['Coefficients'])], axis = 1)" to print the beta coefficients of each variable from the diabetes dataset.</font></p>

# L1 Norm: Lasso

In [ ]:
# Write Python code here:



# L2 Norm: Ridge

In [ ]:
# Write Python code here:



<b> Write the answer here:</b>
#####################################################################################################################

We can see that large values of C give more freedom to the model. Conversely, smaller values of C constrain the model more.

#####################################################################################################################



---





---



## 5.2 Making predictions and using 'accuracy' to evaluate the model

We now have two model that can predict readmission based on the feature variables.

### <font color='blue'> Question 2: Write the Python code to make predictions using the test set with the model that we fitted previously. Make sure you save the predictions as `y_pred`, as this variable will be used later on. </font>

<p><font color='green'> Tip:  You can use the following structure: "y_pred_Model1 = ModelName.predict(X_test_scaled)"</font></p>

In [ ]:
# Write Python code here for "LASSO"



In [ ]:
# Write Python code here for "RIDGE"





---





---



### <font color='blue'> Question 3: Compute the accuracy of both models 'Log_Reg_L1' and 'Log_Reg_L2', and give your opinion about it.
<p><font color='green'> Tip:  You can use the following structure: "round(ModelName.score(X_test_scaled,y_test),3)"</font></p>

In [ ]:
# Write Python code here for "LASSO"
# Use score method to get accuracy of model



In [ ]:
# Write Python code here for "RIDGE"
# Use score method to get accuracy of model



<b> Write the answer here:</b>
#####################################################################################################################

(Double-click here)


#####################################################################################################################



---





---



### Important note regarding accuracy:

Does the accuracy look like a good resut? But be careful...

In [ ]:
# Insert your comments and explanations

print(hospital.readmission.value_counts('no'))

We know that readmission = NO accounts for approximately 83% of the records in the data set. This means that if we set a model to always predict NO, we will automatically obtain 83% model accuracy! This highlights the flaw in using accuracy as the only performance metric, particularly for response variables that are imbalanced (many more of one level than the other). Let's look at the confusion matrix.



---





---



### 5.2.1 Confusion Matrix


In [ ]:
# Insert your comments and explanations

from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Insert your comments and explanations

# Confusion matrix for model using 'LASSO'

# Confusion Matrix
confusion_L1 = metrics.confusion_matrix(y_test, y_pred_L1)

# Visualising the confusion matrix of our KNN model
labels = {'No', 'Yes'}
ax= plt.subplot()
sns.heatmap(confusion_L1, annot=True, fmt='.0f', ax= ax, cmap="viridis")

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['No', 'Yes']); ax.yaxis.set_ticklabels(['No', 'Yes'])


In [ ]:
# Insert your comments and explanations

# Confusion matrix for model using 'RIDGE'

confusion_L2 = metrics.confusion_matrix(y_test, y_pred_L2)

# Visualising the confusion matrix of our KNN model
labels = {'No', 'Yes'}
ax= plt.subplot()
sns.heatmap(confusion_L2, annot=True, fmt='.0f', ax= ax, cmap="viridis")

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix');
ax.xaxis.set_ticklabels(['No', 'Yes']); ax.yaxis.set_ticklabels(['No', 'Yes'])


The confusion matrix shows that only very few records were classified as YES, and almost all of the records were classified as NO.



---





---



### <font color='blue'> Question 4: Explain if you see any differences between both confusion matrix in your own words</font>

<b> Write the answer here:</b>
#####################################################################################################################

(Double-click here)


#####################################################################################################################



---





---



## 5.3 Evaluating the model using F1 Score - L1-norm (LASSO)

In order to assess our model with a more suitable measure, we use the F1 score.

This is a very nice reading about evaluation metrics or performance measures:
https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/

F1 scores need 'YES' to be 1 and 'NO' to be 0.

We build a new list in which 'YES' will be 1 and 'NO' will be 0:

In [ ]:
# Insert your comments and explanations

from sklearn.metrics import f1_score
round(f1_score(y_test, y_pred_L1, pos_label='yes', average='binary'),3)

The F1 score reaches its best value at 1 (perfect precision and recall) and worst at 0. As we can see, our predictive algorithm is not bad.



---





---



### <font color='blue'> Question 5: Repeat the process of evaluating the model using F1 Score, but for our logistic regression model using L2-norm regularization (RIDGE) </font>

In [ ]:
# Insert your comments and explanations





---





---



### <font color='blue'> Question 6: Use the 'classification_report' to list precision, recall, f1 score and support for each class (page 284 and 285 of Book 1). Repeat the process for both models.</font>

In [ ]:
# Insert your comments and explanations

# Write Python code here. TIP: Use y_pred_binary_L1 for LASSO



In [ ]:
# Insert your comments and explanations

# Write Python code here. TIP: Use y_pred_binary_L2 for RIDGE





---



In [ ]:
from sklearn.linear_model import LogisticRegression
Log_Reg_L1 = LogisticRegression(C = 0.01 , penalty = 'l1', solver='liblinear').fit(X_train_scaled, y_train.ravel())
Log_Reg_L2 = LogisticRegression(C = 0.01 , penalty = 'l2',solver='liblinear').fit(X_train_scaled, y_train.ravel())

#Log_Reg_L1.coef_                  #or

coefficients_Log_Reg_L1 = pd.concat([pd.DataFrame(X.columns,columns=['Features']),
                                     pd.DataFrame(np.transpose(Log_Reg_L1.coef_),columns=['Coefficients'])], axis = 1)

# Sorting new DataFrame by Coefficients (Sort Descending)
coefficients_Log_Reg_L1 = coefficients_Log_Reg_L1.sort_values(by='Coefficients', ascending=False)
coefficients_Log_Reg_L1

# check how many coefficients are different from zero
sum(coefficients_Log_Reg_L1.Coefficients!=0)

# check how many coefficients are exactly zero
sum(coefficients_Log_Reg_L1.Coefficients==0)

#Log_Reg_L2.coef_           #or

coefficients_Log_Reg_L2 = pd.concat([pd.DataFrame(X.columns,columns=['Features']),
                                     pd.DataFrame(np.transpose(Log_Reg_L2.coef_),columns=['Coefficients'])], axis = 1)

# Sorting new DataFrame by Coefficients (Sort Descending)
coefficients_Log_Reg_L2 = coefficients_Log_Reg_L2.sort_values(by='Coefficients', ascending=False)
coefficients_Log_Reg_L2

# check how many coefficients are different from zero
sum(coefficients_Log_Reg_L2.Coefficients!=0)

# check how many coefficients are exactly zero
sum(coefficients_Log_Reg_L2.Coefficients==0)

# Making predictions and using 'accuracy' to evaluate the model
y_pred_train_L1  = Log_Reg_L1.predict(X_train_scaled)
y_pred_test_L1   = Log_Reg_L1.predict(X_test_scaled)
# Sanity Check
print(y_pred_train_L1)
print(y_pred_test_L1)

y_pred_train_L2 = Log_Reg_L2.predict(X_train_scaled)
y_pred_test_L2  = Log_Reg_L2.predict(X_test_scaled)
# Sanity Check
print(y_pred_train_L2)
print(y_pred_test_L2)

from sklearn.metrics import accuracy_score
# sklearn.metrics.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)

accuracy_train_L1_alt_sol = accuracy_score(y_train, y_pred_train_L1)
print("The accuracy in the training set for L1 (Lasso) is =", accuracy_train_L1_alt_sol)
accuracy_test_L1_alt_sol = accuracy_score(y_test, y_pred_test_L1)
print("The accuracy in the test set  for L1 (Lasso) is =", accuracy_test_L1_alt_sol)


accuracy_train_L2_alt_sol = accuracy_score(y_train, y_pred_train_L2)
print("The accuracy in the training set for L2 (Ridge) is =", accuracy_train_L2_alt_sol)
accuracy_test_L2_alt_sol = accuracy_score(y_test, y_pred_test_L2)
print("The accuracy in the test set  for L2 (Ridge) is =", accuracy_test_L2_alt_sol)

print(hospital.readmission.value_counts('no'))
# We know that readmission = NO accounts for approximately 83% of the records in the data set. 
# This means that if we set a model to always predict NO, we will automatically obtain 83% model accuracy! 
# This highlights the flaw in using accuracy as the only performance metric, particularly for response variables that are imbalanced (many more of one level than the other). 

# Let's look at the confusion matrix.
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt

# Confusion matrix for model using 'LASSO' using the training set
confusion_L1_train = metrics.confusion_matrix(y_train, y_pred_train_L1)

# Visualising the confusion matrix of our KNN model
labels = {'No', 'Yes'}
ax= plt.subplot()
sns.heatmap(confusion_L1_train, annot=True, fmt='.0f', ax= ax, cmap="viridis")

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix using the training set for L1');
ax.xaxis.set_ticklabels(['No', 'Yes']); ax.yaxis.set_ticklabels(['No', 'Yes'])

# Confusion matrix for model using 'LASSO' using the test set
confusion_L1_test = metrics.confusion_matrix(y_test, y_pred_test_L1)

# Visualising the confusion matrix of our KNN model
labels = {'No', 'Yes'}
ax= plt.subplot()
sns.heatmap(confusion_L1_test, annot=True, fmt='.0f', ax= ax, cmap="viridis")

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix using the test set for L1');
ax.xaxis.set_ticklabels(['No', 'Yes']); ax.yaxis.set_ticklabels(['No', 'Yes'])


# Confusion matrix for model using 'RIDGE' using the training set
confusion_L2_train = metrics.confusion_matrix(y_train, y_pred_train_L2)

# Visualising the confusion matrix of our KNN model
labels = {'No', 'Yes'}
ax= plt.subplot()
sns.heatmap(confusion_L2_train, annot=True, fmt='.0f', ax= ax, cmap="viridis")

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix using the training set for L2');
ax.xaxis.set_ticklabels(['No', 'Yes']); ax.yaxis.set_ticklabels(['No', 'Yes'])


# Confusion matrix for model using 'RIDGE' using the test set
confusion_L2_test = metrics.confusion_matrix(y_test, y_pred_test_L2)

# Visualising the confusion matrix of our KNN model
labels = {'No', 'Yes'}
ax= plt.subplot()
sns.heatmap(confusion_L2_test, annot=True, fmt='.0f', ax= ax, cmap="viridis")

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
ax.set_title('Confusion Matrix using the test set for L2');
ax.xaxis.set_ticklabels(['No', 'Yes']); ax.yaxis.set_ticklabels(['No', 'Yes'])


# Evaluating the model using F1 Score
from sklearn.metrics import f1_score
# F1 for the training and test sets L1
f1_train_L1 = f1_score(y_train, y_pred_train_L1, pos_label='yes', average='binary')
print("F1 for the training set for L1",f1_train_L1)

f1_test_L1 = f1_score(y_test, y_pred_test_L1, pos_label='yes', average='binary')
print("F1 for the test  set for L1",f1_test_L1)

# F1 for the training and test sets L2
f1_train_L2 = f1_score(y_train, y_pred_train_L2, pos_label='yes', average='binary')
print("F1 for the training set for L2",f1_train_L2)

f1_test_L2 =f1_score(y_test,y_pred_test_L2 , pos_label='yes', average='binary')
print("F1 for the test  set for L2",f1_test_L2)

# Use the `classification_report` to list precision, recall, f1 score and support for each class (check the classification report method in the scikit-learn API).
from sklearn.metrics import classification_report

print("Classification report for the training set for L1")
print(classification_report(y_train, y_pred_train_L1))

print("Classification Report for the test set for L1")
print(classification_report(y_test, y_pred_test_L1))

print("Classification report for the training set for L2")
print(classification_report(y_train, y_pred_train_L2))

print("Classification report for the test set for L2")
print(classification_report(y_test, y_pred_test_L2))



---

